<a href="https://colab.research.google.com/github/Abhi10699/google-colab-playground/blob/main/mistral_7b_topic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
locale.getdefaultlocale()

('en_US', 'UTF-8')

In [2]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install bitsandbytes accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 31.5 MB/s eta 0:00:00


In [2]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = transformers.BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
  model_id,
  trust_remote_code=True,
  device_map='auto',
  quantization_config=bnb_config,
)

tokenizer = transformers.AutoTokenizer.from_pretrained(
  model_id,
)


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [3]:
device = 'cuda'

In [4]:
messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
model_inputs = encodeds.to(device)
generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True,pad_token_id=tokenizer.pad_token_id)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST] Certainly! Here's a classic recipe for homemade mayonnaise. This recipe makes about 1 cup (240 ml) of mayonnaise.

Ingredients:
- 1 cup (240 ml) vegetable oil, preferably a neutral-tasting oil such as canola or safflower oil
- 1 egg yolk
- 1 tbsp (15 ml) red wine vinegar or apple cider vinegar
- 1 tbsp (15 ml) Dijon mustard
- 1 tbsp (15 ml) lemon juice
- 1/2 tsp (2.5 g) salt
- 1/2 tsp (2.5 g) ground black pepper

Instructions:
1. In a large, clean bowl, whisk together the egg yolk, vinegar, mustard, and lemon juice.
2. Whisking constantly, slowly drizzle in 1/4 cup (60 ml) of the oil in a thin, steady stream.
3. Once the first 1/4 cup (60 ml) of oil has been completely incorporated, continue to whisk and slowly drizzle in the remaini

In [67]:
def generate_topic_name(tweets):

  topic_list = ""
  for idx, tweet in enumerate(tweets):
    topic_list+= f"\n\n#{idx+1} - {tweet}"

  messages = [
      {"role": "user", "content": "Hi! How are you??"},
      {"role": "assistant", "content": "Hey,I am doing fine!! Thanks for asking"},
      {"role": "user","content": f"You are a very intelligent assistant.You will be provided with {len(tweets)} twitter tweets .Your job is to read those tweets carefully and find common patterns and comeup with a short topic name.If topics are overlapping try to codense them into 1 SINGLE TITLE.Do not let caution limit your creativity. Just keep the topic name in your response and do not respond with anything else."},
      {"role": "assistant","content": "Sure! Provide me with list of tweets and i'll generate a relevent topic label for you!"},
      {"role": "user", "content": f"Here's a list of tweets:  {topic_list}"},
  ]

  encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
  model_inputs = encodeds.to(device)
  generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
  decoded = tokenizer.batch_decode(generated_ids)
  return decoded[0]

In [56]:
import pandas as pd

In [57]:
df = pd.read_csv("/content/top_10_covid_topics_for_llm_6_10.csv")
df.head()

,Unnamed: 0,topic,tweets
0,0,0,never really appreciated before that going to ...
1,1,0,"whether you buy groceries online or in stores,..."
2,2,0,". + andrew whitworth are each donating $250,00..."
3,3,0,"if trump sends a $1,000 relief check then i al..."
4,4,0,"we need teachers, nurses, grocery store worker..."


## Topic For Cluster 0

In [68]:
tweets_0 = list(df[df['topic'] == 0]['tweets'].values)
print(generate_topic_name(tweets_0))
# df.loc[df['topic'] == 0, 'topic_label'] = "Grocery Shopping Challenges during Pandemic"

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Hi! How are you?? [/INST]Hey,I am doing fine!! Thanks for asking</s> [INST] You are a very intelligent assistant.You will be provided with 10 twitter tweets .Your job is to read those tweets carefully and find common patterns and comeup with a short topic name.If topics are overlapping try to codense them into 1 SINGLE TITLE.Do not let caution limit your creativity. Just keep the topic name in your response and do not respond with anything else. [/INST]Sure! Provide me with list of tweets and i'll generate a relevent topic label for you!</s> [INST] Here's a list of tweets:  

#1 - never really appreciated before that going to the supermarket would be such an exciting activity of my day ? oh well.

#2 - whether you buy groceries online or in stores, there are some simple steps you can take to try to limit your exposure to the novel  read:

#3 - . + andrew whitworth are each donating $250,000 to the  to help fund 2 million meals for angelenos in need.

#4 - if trump sends a $1

In [69]:
tweets_1 = list(df[df['topic'] == 1]['tweets'].values)
print(generate_topic_name(tweets_1))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Hi! How are you?? [/INST]Hey,I am doing fine!! Thanks for asking</s> [INST] You are a very intelligent assistant.You will be provided with 10 twitter tweets .Your job is to read those tweets carefully and find common patterns and comeup with a short topic name.If topics are overlapping try to codense them into 1 SINGLE TITLE.Do not let caution limit your creativity. Just keep the topic name in your response and do not respond with anything else. [/INST]Sure! Provide me with list of tweets and i'll generate a relevent topic label for you!</s> [INST] Here's a list of tweets:  

#1 - worried about empty shelves or food and supply hoarding during the  outbreak?there is enough to go around despite the high demand, officials and grocers say. shelves will be restocked.

#2 - dear hoarder :?19

#3 - live streams? sit in your car at grocery store and watch patterns of who goes in/out and how much they get and if theyâre wearing protective devices. interactions on social media about 

In [70]:
tweets_2 = list(df[df['topic'] == 2]['tweets'].values)
print(generate_topic_name(tweets_2))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Hi! How are you?? [/INST]Hey,I am doing fine!! Thanks for asking</s> [INST] You are a very intelligent assistant.You will be provided with 10 twitter tweets .Your job is to read those tweets carefully and find common patterns and comeup with a short topic name.If topics are overlapping try to codense them into 1 SINGLE TITLE.Do not let caution limit your creativity. Just keep the topic name in your response and do not respond with anything else. [/INST]Sure! Provide me with list of tweets and i'll generate a relevent topic label for you!</s> [INST] Here's a list of tweets:  

#1 - check out a covid-19 cure for restaurants: reaching older diners  via

#2 - a small excerpt from a good article: "...the coronavirus is triggering a massive change in the way movies are released, which could have a lasting effect on how people consume hollywood entertainment"

#3 - large retailers who hike prices of necessary products should be warned and prosecuted

#4 - covid-19: all stores in om

In [71]:
tweets_3 = list(df[df['topic'] == 3]['tweets'].values)
print(generate_topic_name(tweets_3))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Hi! How are you?? [/INST]Hey,I am doing fine!! Thanks for asking</s> [INST] You are a very intelligent assistant.You will be provided with 10 twitter tweets .Your job is to read those tweets carefully and find common patterns and comeup with a short topic name.If topics are overlapping try to codense them into 1 SINGLE TITLE.Do not let caution limit your creativity. Just keep the topic name in your response and do not respond with anything else. [/INST]Sure! Provide me with list of tweets and i'll generate a relevent topic label for you!</s> [INST] Here's a list of tweets:  

#1 - put the prices up and then when you try and call them to discuss, they state that they arenât taking any billing calls due to ! outrageous. time for  and

#2 - this guy is hilarious ????

#3 - never in my life would i think i would see queues for the supermarket honestly - whoâs parents are doing this?   because surely everyone is denying in but i know some of your parents have been doing it. so 

In [72]:
tweets_4 = list(df[df['topic'] == 4]['tweets'].values)
print(generate_topic_name(tweets_4))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Hi! How are you?? [/INST]Hey,I am doing fine!! Thanks for asking</s> [INST] You are a very intelligent assistant.You will be provided with 10 twitter tweets .Your job is to read those tweets carefully and find common patterns and comeup with a short topic name.If topics are overlapping try to codense them into 1 SINGLE TITLE.Do not let caution limit your creativity. Just keep the topic name in your response and do not respond with anything else. [/INST]Sure! Provide me with list of tweets and i'll generate a relevent topic label for you!</s> [INST] Here's a list of tweets:  

#1 - the only aisle in the  not out of

#2 - dear boris people are stockpiling food shopping for groceries is becoming a problem if you could reassure the public and calm the panic that would be good

#3 - and warning every medical and departmental stores who take chance to increase the prices of hygienic products during this time and hereby also we are bound to thank every and employees at hospitals wh

In [73]:
tweets_4 = list(df[df['topic'] == 5]['tweets'].values)
print(generate_topic_name(tweets_4))
# Impact of Panic Buying and Challenges Faced by Grocery Store Employees

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Hi! How are you?? [/INST]Hey,I am doing fine!! Thanks for asking</s> [INST] You are a very intelligent assistant.You will be provided with 10 twitter tweets .Your job is to read those tweets carefully and find common patterns and comeup with a short topic name.If topics are overlapping try to codense them into 1 SINGLE TITLE.Do not let caution limit your creativity. Just keep the topic name in your response and do not respond with anything else. [/INST]Sure! Provide me with list of tweets and i'll generate a relevent topic label for you!</s> [INST] Here's a list of tweets:  

#1 - i work at a grocery store.we wont get any toilet paper for the next 2 week.hope yâall like wiping your ass on a paper plate.

#2 - working from home because of coronavirus and struggling to get hold of  a laptop? why not try refurbished. we can get hold of stock at great prices with warranty. drop us a line for details

#3 - i asked my grandparents-in-law what they need from the grocery store so i